
###### Hidden Markov Models PRIVATE NOTEBOOK


###### Contents in this notebook

######  Hidden markov models learning

- Computing counts from a corpus
- From counts to probabilities: finding the parameters of initial, transition and emission probabilities.
- Smoothing non seen events in avoid overfitting.

######  Working with scores instead of probabilities

- Rewritting probabilities with log probabilities. 
- Why do we want to work with log probabilities? Understand issues when computing a product of probabilities.
- The `logsumexp` trick.

######  The forward backward algorithm

- Computing the probability of a sequence $P(X=x)$
- Computing the probability of a stage given an input sequence $P(Y_i=x \,\vert\, X=x)$. 
- Use this probabilities to do Posterior decoding. Evaluate posterior decoding in a problem.


###### Computing the most likely hidden state sequence: The viterbi algorithm

- How to compute $argmax_{y_1,\dots,y_N \in \Lambda^N} P(Y_{1:N}=y_{1:N}\,\vert\, X=x)$



In [1]:
%load_ext autoreload
%autoreload 2


In [2]:

import scipy
import numpy as np

import os,sys,inspect
currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parentdir = os.path.dirname(currentdir)
sys.path.insert(0,parentdir) 
import skseq

###### classes used to store the sequences

We will use

- class ``Sequence          ----> skseq/sequences/sequence.py`` file
- class ``LabelDictionary   ----> skseq/sequences/label_dictionary.py`` file
- class ``SequenceList      ----> skseq/sequences/sequence_list.py`` file
- class ``_SequenceIterator ----> skseq/sequences/sequence_list.py`` file

# Setting a Sequence Object


Before we start with HMMs let us import a `Sequence` class that we can use to build our model.

- The `Sequence` class is defined in ``skseq/sequences/sequence.py``. 
    - A `Sequence` in a supervised learning problem consist on a set of words and tags associated to words.
    - For example ``w_1/t_1 w_2/t_2 w_3/t_3`` is a sequence of length 3 with words ``w_i`` and tags ``t_i``.
    

In order to instanciate a Sequence we essentially need a list of words and a list of tags of the same size.

In order to do use memory efficiently we will not store strings for the words and tags. We will store integer values to represent words and tags.

An instance of a `Sequence` object has 2 very important fields that store the input and output:

- **.x** attribute: list of words (integer words)

- **.y** attribute: list of tags (integer tags)

Then we need to keep a mapping from integers to words and from integers to tags.


In [3]:
import skseq
import skseq.sequences
import skseq.readers

In [4]:
from skseq.sequences import sequence

seq = skseq.sequences.sequence.Sequence(x=[1,3,2,4],
                                        y=[0,2,1,1])


Note that seq has a fancy print version (code in `def __repr__`) that makes the object be printed
as `w_1/t_1 w_2/t_2 w_3/t_3` when we simply write `seq` in the terminal

In [5]:
seq

1/0 3/2 2/1 4/1 

In [6]:
type(seq)

skseq.sequences.sequence.Sequence

The code, as is written, can get any list as input and any list as output (as long as both have the same length) 

In [7]:
seq = skseq.sequences.sequence.Sequence(x=["David","was","happy"],
                                        y=["P","0","0"])

seq

David/P was/0 happy/0 

In [8]:
seq.x

['David', 'was', 'happy']

In [9]:
seq.y

['P', '0', '0']

# Building a vocabulary and a SequenceList

Given a training set with words and tags we want to build a SequenceList object definded in  ``skseq/sequences/sequence_list.py``.

A  `SequenceList` is a class that is initialized using:

- **dictionary for the words**
- **a dictionary for the tags**
- **an empty `seq_list` where the Sequences read from the data will be stored.**


    class SequenceList(object):

        def __init__(self, x_dict, y_dict):
            self.x_dict = x_dict
            self.y_dict = y_dict
            self.seq_list = []


Let us create 3 sequence list for train, test and validation.  

We will use the conll dataset and the class  ``PostagCorpus``.
The class has a method ``.read_sequence_list_conll`` that will return the **SequenceList** object we want



    def read_sequence_list_conll(self, train_file,
                                 mapping_file=("%s/en-ptb.map"
                                               % dirname(__file__)),
                                 max_sent_len=100000,
                                 max_nr_sent=100000):

        # Build mapping of postags:
        mapping = {}
        if mapping_file is not None:
            for line in open(mapping_file):
                coarse, fine = line.strip().split("\t")
                mapping[coarse.lower()] = fine.lower()

        instance_list = self.read_conll_instances(train_file,
                                                  max_sent_len,
                                                  max_nr_sent,
                                                  mapping)

        seq_list = SequenceList(self.word_dict, self.tag_dict)

        for sent_x, sent_y in instance_list:
            seq_list.add_sequence(sent_x, sent_y,  self.word_dict, self.tag_dict)

        return seq_list

In [10]:
import skseq.readers.pos_corpus
corpus = skseq.readers.pos_corpus.PostagCorpus()

data_path = "../data/conll"

train_seq = corpus.read_sequence_list_conll(data_path + "/train-02-21.conll", 
                                            max_sent_len=100, max_nr_sent=5000)

test_seq = corpus.read_sequence_list_conll(data_path + "/test-23.conll",
                                           max_sent_len=100, max_nr_sent=1000)

dev_seq = corpus.read_sequence_list_conll(data_path + "/dev-22.conll", 
                                          max_sent_len=100, max_nr_sent=1000)

In [11]:
type(train_seq[1])

skseq.sequences.sequence.Sequence

In [12]:
type(train_seq)

skseq.sequences.sequence_list.SequenceList

In [13]:
train_seq.__dict__.keys()

dict_keys(['x_dict', 'y_dict', 'seq_list'])

In [14]:
len(train_seq.x_dict), len(train_seq.y_dict)

(16937, 12)

In [15]:
train_seq.__dict__.keys()

dict_keys(['x_dict', 'y_dict', 'seq_list'])

In [16]:
len(test_seq.x_dict), len(test_seq.y_dict)

(16937, 12)

In [17]:
len(train_seq)

5000

In [18]:
# first sentence
train_seq[0].__dict__.keys()

dict_keys(['x', 'y'])

In [19]:
train_seq[1]

42/2 40/2 43/6 44/2 41/4 

In [20]:
train_seq.__dict__.keys()

dict_keys(['x_dict', 'y_dict', 'seq_list'])

In [21]:
# Set of possible tags Lambda
# train_seq.y_dict is a dictionary of mappings from tag to integer id 
train_seq.y_dict

{'adp': 0,
 'det': 1,
 'noun': 2,
 'num': 3,
 '.': 4,
 'prt': 5,
 'verb': 6,
 'conj': 7,
 'adv': 8,
 'pron': 9,
 'adj': 10,
 'x': 11}

In [22]:
# number of possible words
len(train_seq.x_dict)

16937

In [23]:
train_seq[1]

42/2 40/2 43/6 44/2 41/4 

## Using our corpus ``sequencelist`` to map integers to words

Sequences can use ``SequenceList`` objects to map word_ids and tag_ids to words and tags.

All ``sequence`` objects have the **``.to_words``** method which allows us to print the words given a **``SequenceList``** object. 

In [24]:
train_seq.__dict__.keys()

dict_keys(['x_dict', 'y_dict', 'seq_list'])

In [25]:
sequence = train_seq[1]

In [26]:
sequence

42/2 40/2 43/6 44/2 41/4 

In [27]:
sequence.to_words(sequence_list=train_seq)

'Ms./noun Haag/noun plays/verb Elianti/noun ./. '

##  Hidden markov models




### Using toy data

In [28]:
import skseq.sequences.sequence as seq

In [29]:
Sigma = ["walk", "shop", "clean", "tennis"]
Lambda = ["rainy", "sunny"]

sequence_list = []

In [30]:
# walk/rainy walk/sunny shop/sunny clean/sunny 
# walk/rainy walk/rainy shop/rainy clean/sunny 
# walk/sunny shop/sunny shop/sunny clean/sunny 

s1 = seq.Sequence(["walk", "walk", "shop", "clean"],
                  ["rainy", "sunny", "sunny", "sunny"])

s2 = seq.Sequence(["walk", "walk", "shop", "clean"], 
                  ["rainy", "rainy", "rainy", "sunny"])

s3 = seq.Sequence(["walk", "shop", "shop", "clean"], 
                  ["sunny", "sunny", "sunny", "sunny"])

train_sequences = [s1, s2, s3];

s1_t = seq.Sequence(["walk", "walk", "shop", "clean"], 
                    ["rainy", "sunny", "sunny", "sunny"])

s2_t = seq.Sequence(["clean", "walk", "tennis", "walk"], 
                    ["sunny", "sunny", "sunny", "sunny"])

test_sequences = [s1_t, s2_t];

In [31]:
train_sequences[0]

walk/rainy walk/sunny shop/sunny clean/sunny 

### Training the model 



In [32]:
word_to_pos  = {"walk":0,  "shop":1, "clean":2, "tennis":3}
state_to_pos = {"rainy":0, "sunny": 1}

In [33]:
def update_initial_counts(initial_counts, seq, state_to_pos):    
    """
    TODO: Fill in this function and remove the pass
    """

    pass

def update_transition_counts(transition_counts, seq, state_to_pos):
    """
    TODO: Fill in this function and remove the pass
    """
    pass

def update_emission_counts(emission_counts, seq, state_to_pos, word_to_pos):
    """
    TODO: Fill in this function and remove the pass
    """

def update_final_counts(final_counts, seq, state_to_pos):
    """
    TODO: Fill in this function and remove the pass
    """


Using the previous function we can train the emisssion, initial and transition probabilities by simply counting.

In [34]:
def sufficient_statistics_hmm(sequences, state_to_pos, word_to_pos):
    
    n_states = len(state_to_pos)
    n_words  = len(word_to_pos)
    initial_counts      = np.zeros((n_states))
    transition_counts   = np.zeros((n_states, n_states))
    final_counts        = np.zeros((n_states))
    emission_counts     = np.zeros((n_states, n_words))
    
    for seq in sequences:
        update_initial_counts(initial_counts, seq, state_to_pos)
        update_transition_counts(transition_counts, seq,  state_to_pos)
        update_emission_counts(emission_counts, seq,  state_to_pos, word_to_pos) 
        update_final_counts(final_counts, seq,  state_to_pos) 
    
    return initial_counts, transition_counts, final_counts, emission_counts
    

In [35]:
counts = sufficient_statistics_hmm(train_sequences, 
                                   state_to_pos,
                                   word_to_pos);

In [36]:
initial_counts, transition_counts, final_counts, emission_counts = counts;

In [37]:
initial_counts

array([2., 1.])

In [38]:
transition_counts

array([[2., 0.],
       [2., 5.]])

In [39]:
final_counts

array([0., 3.])

In [40]:
emission_counts.T

array([[3., 2.],
       [1., 3.],
       [0., 3.],
       [0., 0.]])

In [41]:
word_to_pos

{'walk': 0, 'shop': 1, 'clean': 2, 'tennis': 3}

In [42]:
state_to_pos

{'rainy': 0, 'sunny': 1}

### Sanity Checks HMM

- Initial counts must sum to the number of sentences  $$ \sum_{k=1}^K C_{\text{init}}(c_k) = M$$

- Transition counts and Final Counts should sum to the number of tokens: $$\sum_{k,l=1}^K C_{\text{trans}}(c_k,c_l)  + \sum_{k=1}^K C_{\text{final}}(c_k) = M \cdot N$$

- Emission counts must sum to the number of tokens
$$
\sum_{j=1}^J \sum_{k=1}^K C_{\text{emiss}}(w_j,c_k) = M \cdot N 
$$

In [43]:
M = len(train_sequences)
N = len(train_sequences[0].x)
print("M:\t", M, "\nN:\t", N,"\nM*N:\t", M*N)

M:	 3 
N:	 4 
M*N:	 12


In [44]:
print("initial_counts sum: ", np.sum(initial_counts))
print("emission_counts sum: ", np.sum(emission_counts))
print("transition and final counts sum: ",\
       np.sum(transition_counts) + sum(final_counts))

initial_counts sum:  3.0
emission_counts sum:  12.0
transition and final counts sum:  12.0


In [45]:
initial_counts

array([2., 1.])

In [46]:
transition_counts

array([[2., 0.],
       [2., 5.]])

In [47]:
final_counts

array([0., 3.])

In [48]:
emission_counts

array([[3., 1., 0., 0.],
       [2., 3., 3., 0.]])

### From counts to probabilities

The following formulas specify how to find the parameters of the HMM:

$$
P_{\text{init}}(c_k \,\vert\, \text{start}) = \frac{C_{\text{init}}(c_k)}{ \sum_{k=1}^K
C_{\text{init}} (c_l)}
$$

$$
P_{\text{final}}(\text{stop} \,\vert\, c_l) = \frac{C_{\text{final}}(c_l) }
{\sum_{k=1}^K C_{\text{trans}}(c_k,c_l) + C_{\text{final}}(c_l)}
$$

$$
P_{\text{trans}}( c_k \,\vert\, c_l) = \frac{C_{\text{trans}}(c_k, c_l) }
{\sum_{p=1}^K C_{\text{trans}}(c_p,c_l) + C_{\text{final}}(c_l)}
$$

$$
P_{\text{emiss}} (w_j \,\vert\, c_k) = \frac{C_{\text{emiss}} (w_j, c_k) }{\sum_{q=1}^J C_{\text{emiss}}(w_q,c_k)}
$$

In [49]:
#initial_probs    = (initial_counts / np.sum(initial_counts))
#transition_probs = transition_counts/(np.sum(transition_counts, 1, keepdims=True) + final_counts)
#final_probs      = final_counts/(np.sum(transition_counts, 1, keepdims=True) + final_counts )
#emission_probs   = emission_counts / np.sum(emission_counts, 1, keepdims=True)

    
initial_probs    = (initial_counts / np.sum(initial_counts))
transition_probs = transition_counts/(np.sum(transition_counts,0) + final_counts)
final_probs      = final_counts/(np.sum(transition_counts, 0) + final_counts )
emission_probs   = emission_counts.T / np.sum(emission_counts, 1)

print("\ninitial_probs")
print(initial_probs)

print("\ntransition_probs")
print(transition_probs)

print("\nfinal_probs")
print(final_probs)

print("\nemission_probs")
print(emission_probs)


initial_probs
[0.66666667 0.33333333]

transition_probs
[[0.5   0.   ]
 [0.5   0.625]]

final_probs
[0.    0.375]

emission_probs
[[0.75  0.25 ]
 [0.25  0.375]
 [0.    0.375]
 [0.    0.   ]]


In [50]:
word_to_pos

{'walk': 0, 'shop': 1, 'clean': 2, 'tennis': 3}


OBSERVATION:

**If we stack trainsition and final counts and normalize them we get
a proper conditional probability distribution**

In [51]:
transitions_with_final_counts = np.vstack((transition_counts,
                                           final_counts))

In [52]:
transitions_with_final_counts 

array([[2., 0.],
       [2., 5.],
       [0., 3.]])

In [53]:
transitions_with_final_counts/ np.sum(transitions_with_final_counts,0)

array([[0.5  , 0.   ],
       [0.5  , 0.625],
       [0.   , 0.375]])

## Working with scores 


In [54]:
#a = np.random.rand([0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,0.10])
a = np.array([0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,0.10])

print(np.log(sum(np.exp(a))))
print(np.log(sum(np.exp(10*a))))
print(np.log(sum(np.exp(100*a))))
print(np.log(sum(np.exp(1000*a))))

2.7999098843442183
9.45876378447825
90.00004540096037
inf


/Users/davidbuchaca1/anaconda3/envs/py3_7/lib/python3.6/site-packages/ipykernel_launcher.py:7: RuntimeWarning: overflow encountered in exp
  import sys


In [55]:
def logzero():
    return -np.inf


def safe_log(x):
    print(x)
    if x == 0:
        return logzero()
    return np.log(x)


def logsum_pair(logx, logy):
    """
    Return log(x+y), avoiding arithmetic underflow/overflow.

    logx: log(x)
    logy: log(y)

    Rationale:

    x + y    = e^logx + e^logy
             = e^logx (1 + e^(logy-logx))
    log(x+y) = logx + log(1 + e^(logy-logx)) (1)

    Likewise,
    log(x+y) = logy + log(1 + e^(logx-logy)) (2)

    The computation of the exponential overflows earlier and is less precise
    for big values than for small values. Due to the presence of logy-logx
    (resp. logx-logy), (1) is preferred when logx > logy and (2) is preferred
    otherwise.
    """
    if logx == logzero():
        return logy
    elif logx > logy:
        return logx + np.log1p(np.exp(logy-logx))
    else:
        return logy + np.log1p(np.exp(logx-logy))


def logsum(logv):
    """
    Return log(v[0]+v[1]+...), avoiding arithmetic underflow/overflow.
    """
    res = logzero()
    for val in logv:
        res = logsum_pair(res, val)
    return res

In [56]:
import numpy as np

#a = np.random.rand([0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,0.10])
a = np.array([0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,0.10])

print(np.log(sum(np.exp(a))))
print(np.log(sum(np.exp(10*a))))
print(np.log(sum(np.exp(100*a))))
print(np.log(sum(np.exp(1000*a))))

print("\n")
print(logsum(a))
print(logsum(10*a))
print(logsum(100*a))
print(logsum(1000*a))

2.7999098843442183
9.45876378447825
90.00004540096037
inf


2.7999098843442183
9.45876378447825
90.00004540096037
900.0


/Users/davidbuchaca1/anaconda3/envs/py3_7/lib/python3.6/site-packages/ipykernel_launcher.py:9: RuntimeWarning: overflow encountered in exp
  if __name__ == '__main__':


In [57]:
import numpy as np

def logsumexp(vec):
    c = np.max(vec)
    return c + np.log(np.sum(np.exp(vec-c)))

#a = np.random.rand([0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,0.10])
a = np.array([0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,0.10])

print(np.log(sum(np.exp(a))))
print(np.log(sum(np.exp(10*a))))
print(np.log(sum(np.exp(100*a))))
print(np.log(sum(np.exp(1000*a))))

print("\n")
print(logsumexp(a))
print(logsumexp(10*a))
print(logsumexp(100*a))
print(logsumexp(1000*a))

2.7999098843442183
9.45876378447825
90.00004540096037
inf


2.7999098843442183
9.45876378447825
90.00004540096037
900.0


/Users/davidbuchaca1/anaconda3/envs/py3_7/lib/python3.6/site-packages/ipykernel_launcher.py:13: RuntimeWarning: overflow encountered in exp
  del sys.path[0]


In [58]:
# How can we find this? probabilities will be between 0 and 1

## HMM Class

In [59]:
class HMM(object):
    
    def __init__(self, word_to_pos={}, state_to_pos={}):
        self.fitted = False
        self.counts = {"emission": None, "transition":None, "final":None, "initial":None}
        self.probs  = {"emission": None, "transition":None, "final":None, "initial":None}
        self.scores = {"emission": None, "transition":None, "final":None, "initial":None}
        self.decode = set(["posterior", "viterbi"])
        self.word_to_pos  = word_to_pos
        self.state_to_pos = state_to_pos
        self.pos_to_word  = {v: k for k, v in word_to_pos.items()}
        self.pos_to_state = {v: k for k, v in state_to_pos.items()}
    
        self.n_states     = len(state_to_pos)
        self.n_words      = len(word_to_pos)
        self.fitted = False

    def fit(self, observation_lables: list, state_labels: list):
        """
        Computes and saves: counts, probs, scores.
        """
        if self.state_to_pos is None or self.word_to_pos is None:
            print("Error state_to_pos or word_to_pos needed to be defined")
            return
            
        self.counts = self.sufficient_statistics_hmm(observation_lables, state_labels)       
        self.probs  = self.compute_probs(self.counts)  
        self.scores = self.compute_scores(self.probs)  
        self.fitted = True
        
    def sufficient_statistics_hmm(self, observation_lables, state_labels):

        state_to_pos, word_to_pos = self.state_to_pos, self.word_to_pos
        
        def update_initial_counts(initial_counts, seq_x, state_to_pos):
            """
            TODO: Fill in this function and remove the pass
            """
            pass

            
        def update_transition_counts(transition_counts, seq_y, state_to_pos):
            """
            TODO: Fill in this function and remove the pass
            """
            pass


        def update_emission_counts(emission_counts, seq_x, seq_y, state_to_pos, word_to_pos):
            """
            TODO: Fill in this function and remove the pass
            """
            pass

        def update_final_counts(final_counts, seq_y, state_to_pos):
            """
            TODO: Fill in this function and remove the pass
            """
            pass


        n_states = len(state_to_pos)
        n_words  = len(word_to_pos)
        initial_counts      = np.zeros((n_states))
        transition_counts   = np.zeros((n_states, n_states))
        final_counts        = np.zeros((n_states))
        emission_counts     = np.zeros((n_states, n_words))

        for seq_x, seq_y in zip(observation_lables, state_labels):
            update_initial_counts(initial_counts, seq_y, state_to_pos)
            update_transition_counts(transition_counts, seq_y,  state_to_pos)
            update_emission_counts(emission_counts, seq_x, seq_y, state_to_pos, word_to_pos) 
            update_final_counts(final_counts, seq_y,  state_to_pos) 

        return {"emission":   emission_counts, 
                "transition": transition_counts,
                "final":      final_counts, 
                "initial":    initial_counts}
    
    def compute_probs(self, counts):
        
        initial_counts    = counts['initial']
        transition_counts = counts['transition']
        emission_counts   = counts['emission']
        final_counts      = counts['final']

        initial_probs    = (initial_counts / np.sum(initial_counts))
        transition_probs = transition_counts/(np.sum(transition_counts,0) + final_counts)
        final_probs      = final_counts/(np.sum(transition_counts, 0) + final_counts )
        emission_probs   = (emission_counts.T / np.sum(emission_counts, 1)).T
    
        return {"emission":   emission_probs, 
                "transition": transition_probs,
                "final":      final_probs, 
                "initial":    initial_probs}
    
    def compute_scores(self, probs):
         return {"emission":   np.log(probs["emission"]), 
                 "transition": np.log(probs["transition"]),
                 "final":      np.log(probs["final"]), 
                 "initial":    np.log(probs["initial"])}
        
    def forward_computations(self, x: list):
        forward_x = None
        return forward_x
    
    def backward_computations(self, x:list):
        backward_x = None
        return backward_x
    
    def log_forward_computations(self, x: list):
        """
        Compute the log_forward computations

        Assume there are S possible states and a sequence of length N.
        This method will compute iteritavely the log_forward quantities.

        * log_f is a S x N Array.
        * log_f_x[:,i] will contain the forward quantities at position i.
        * log_f_x[:,i] is a vector of size S.
        
        Returns
        - log_f_x: Array of size K x N
        """ 
        n_x = len(x)
        
        # log_f_x initialized to -Inf because log(0) = -Inf
        log_f_x = np.zeros((self.n_states, n_x)) - np.Inf
        x_emission_scores = np.array([hmm.scores['emission'][:, hmm.word_to_pos[w]] for w in x]).T
        
        log_f_x[:,0] = x_emission_scores[:, 0] + self.scores['initial']
        
        """
        TODO: Fill in this function and remove the pass
        compute log_f_x, log_likelihood
        """
        pass


        return log_f_x, log_likelihood
    
    
    def log_backward_computations(self, x: list):
        n_x = len(x)
        
        # log_f_x initialized to -Inf because log(0) = -Inf
        log_b_x = np.zeros((self.n_states, n_x)) - np.Inf
        x_emission_scores = np.array([hmm.scores['emission'][:, hmm.word_to_pos[w]] for w in x]).T
        log_b_x[:,-1] = self.scores['final']

        """
        TODO: Fill in this function and remove the pass
        compute log_b_x, log_likelihood
        """
        
        return log_b_x, log_likelihood
        
    def predict_labels(self, x: list, decode="posterior"):
        """
        Retuns a sequence of states for each word in **x**.
        The output depends on the **decode** method chosen.
        """
        assert decode in self.decode, "decode `{}` is not valid".format(decode)
        
        if decode is 'posterior':
            return self.posterior_decode(x)
        
        if decode is 'viterbi':
            return self.viterbi_decode(x)

    def compute_state_posteriors(self, x:list):
        log_f_x, log_likelihood = self.log_forward_computations(x)
        log_b_x, log_likelihood = self.log_backward_computations(x)
        state_posteriors = np.zeros((self.n_states, len(x)))
        
        for pos in range(len(x)):
            state_posteriors[:, pos] = log_f_x[:, pos] + log_b_x[:, pos] - log_likelihood
        return state_posteriors

    def posterior_decode(self, x: list, decode_states=True):
        
        state_posteriors = self.compute_state_posteriors(x)
        y_hat = state_posteriors.argmax(axis=0)
        
        if decode_states:
            y_hat = [hmm.pos_to_state[y] for y in y_hat]
            
        return y_hat

In [60]:
hmm = HMM(word_to_pos, state_to_pos)

In [61]:
hmm.state_to_pos, hmm.word_to_pos

({'rainy': 0, 'sunny': 1}, {'walk': 0, 'shop': 1, 'clean': 2, 'tennis': 3})

In [62]:
X = [t.x for t in train_sequences]
Y = [t.y for t in train_sequences]

In [63]:
hmm = HMM(word_to_pos, state_to_pos)
hmm.fit(X, Y)


/Users/davidbuchaca1/anaconda3/envs/py3_7/lib/python3.6/site-packages/ipykernel_launcher.py:85: RuntimeWarning: divide by zero encountered in log
/Users/davidbuchaca1/anaconda3/envs/py3_7/lib/python3.6/site-packages/ipykernel_launcher.py:86: RuntimeWarning: divide by zero encountered in log
/Users/davidbuchaca1/anaconda3/envs/py3_7/lib/python3.6/site-packages/ipykernel_launcher.py:87: RuntimeWarning: divide by zero encountered in log


In [64]:
for k in hmm.counts:
    print(k,'\n', hmm.counts[k],'\n')

emission 
 [[3. 1. 0. 0.]
 [2. 3. 3. 0.]] 

transition 
 [[2. 0.]
 [2. 5.]] 

final 
 [0. 3.] 

initial 
 [2. 1.] 



In [65]:
for k in hmm.probs:
    print(k,'\n', hmm.probs[k],'\n')

emission 
 [[0.75  0.25  0.    0.   ]
 [0.25  0.375 0.375 0.   ]] 

transition 
 [[0.5   0.   ]
 [0.5   0.625]] 

final 
 [0.    0.375] 

initial 
 [0.66666667 0.33333333] 



In [66]:
for k in hmm.scores:
    print(k,'\n', hmm.scores[k],'\n')

emission 
 [[-0.28768207 -1.38629436        -inf        -inf]
 [-1.38629436 -0.98082925 -0.98082925        -inf]] 

transition 
 [[-0.69314718        -inf]
 [-0.69314718 -0.47000363]] 

final 
 [       -inf -0.98082925] 

initial 
 [-0.40546511 -1.09861229] 




## Efficient forward probability computation

The forward probability represents the probability that in position
$i$ we are in state $Y_i = c_k$ and that we have observed $x_1,\ldots,x_i$
up to that position. Therefore, its mathematical expression is:
\begin{equation}
\mathbf{Forward \ Probability\!:}\;\;\;\;  \mathrm{forward}(i, c_k) = P(Y_i = c_k, X_1=x_1,\ldots, X_i = x_i)
\end{equation}


Using the independence assumptions of the HMM we can compute $\mathrm{forward}(i, c_k)$ using all the forward computations \{$\mathrm{forward}(i -1, c)$ for $c \in \Lambda$\}. In order to facilitate the notation of the following argument we will denote by $x_{i:j}$  the assignemnt $X_i = x_i, \dots, X_j = x_j$. Therefore we can write   $\mathrm{forward}(i, y_i) $ as $P( y_i, x_{1:i } ) $ and rewrite the forward expression as follows:

\begin{equation}
  P( y_i, x_{1:i } ) =  \sum_{y_{i-1} \in \Lambda} P( y_i ,y_{i-1}, x_{1:i } )  =  \sum_{y_{i-1} \in \Lambda} P( x_i  | y_i,  y_{i-1},  x_{1:i-1 } ) \cdot P(y_i  | y_{i-1},  x_{1:i-1 }) \cdot P(y_{i-1},  x_{1:i-1 })  
\end{equation}


Using the **Observation independence** and the **Independence of previous states** properties of the first order HMM we have $P( x_i  | y_i,  y_{i-1},  x_{1:i-1 } ) = P( x_i  | y_i) $ and $P(y_i  | y_{i-1},  x_{1:i-1 })  = P(y_i  | y_{i-1})  $. Therefore the previous equation can be written, 
for $i \in \{2,\dots,N\}$ (where $N$ is the length of the sequence), as 

\begin{equation}
 \mathrm{forward}(i, y_i)  = \sum_{y_{i-1} \in \Lambda} P( x_i  | y_i, ) \cdot P(y_i  | y_{i-1}) \cdot \mathrm{forward}(i-1, y_{i-1})   
\end{equation}


The previous equation proves that  the forward probability can be defined by the
following recurrence rule: 

\begin{eqnarray}
\mathrm{forward}(1, c_k)&=& P_{\text{init}}(c_k|\text{start}) \times P_{\mathrm{emiss}}(x_1 | c_k)
 \\
 \mathrm{forward}(i, c_k) &=& \left(  \sum_{c_l \in \Lambda} P_{\mathrm{trans}}(c_k | c_l) \times \mathrm{forward}(i-1, c_l) \right) \times P_{\mathrm{emiss}}(x_i | c_k) 
 \\
  \mathrm{forward}(N+1, \text{stop}) &=& \sum_{c_l \in \Lambda} P_{\text{final}}(\text{ stop} | c_l) \times \mathrm{forward}(N, c_l).
\end{eqnarray}


Using the forward trellis one can compute the likelihood simply as:

\begin{equation}
P(X=x) = \mathrm{forward}(N+1, \text{ stop}).
\end{equation}

Although the forward probability is enough to calculate the likelihood of a given sequence, we will also need the backward probability to calculate the state posteriors. 

In [67]:
example = train_sequences[1].x
example

['walk', 'walk', 'shop', 'clean']

In [68]:
log_forward, loglikelihood = hmm.log_forward_computations(example)

In [69]:
log_forward

array([[-0.69314718, -1.67397643, -3.75341798,        -inf],
       [-2.48490665, -2.58334672, -2.94017562, -4.08740307]])

In [70]:
loglikelihood

-5.068232326005127


## Efficient backward probability computation



The backward probability is similar to the forward probability, but operates in the inverse direction.
It represents the probability of observing $x_{i+1},\ldots,x_N$ from position $i+1$ up to $N$, given that at position $i$ we are at state $Y_i = c_l$:

\begin{equation}
\mathbf{Backward \ Probability\!:}\;\;\;\;  \text{backward}(i, c_l) = P(X_{i+1}=x_{i+1},\ldots, X_N=x_N | Y_i = c_l).
\end{equation}



Using the independence assumptions of the HMM we can compute $\text{backward}(i, c_k)$ using all the backward computations $\text{backward}(i +1, c)$ for $c \in \Lambda$.

Therefore we can write   $\text{backward}(i, y_i) $ as $P( x_{i+1:N} | y_i ) $ and rewrite the forward expression as follows:

\begin{equation}
  P( x_{i+1:N} | y_i ) =  \sum_{y_{i+1} \in \Lambda} P( x_{i+1:N}, y_{i+1} | y_i)  =  \sum_{y_{i+1} \in \Lambda} P( x_{i+2:N} | y_i, y_{i+1}, x_{i+1}) 
   P( x_{i+1}, |  y_{i+1},  y_{i}) P( y_{i+1} | y_i)
\end{equation}

Using the previous equation we have proved that the backward probability can be defined by the following recurrence rule:


\begin{eqnarray}
\mathrm{backward}(N, c_l) &=& P_{\text{final}}(\text{stop} | c_l)  \\
\text{backward}(i, c_l) &=&  \displaystyle \sum_{c_k \in \Lambda} P_{\text{trans}}(c_k | c_l) \times 
\text{backward}(i+1, c_k) \times P_{\text{emiss}}(x_{i+1} | c_k) 
 \\
  \mathrm{backward}(0, \text{start}) &=& \sum_{c_k \in \Lambda} P_{\mathrm{init}}(c_k | \text{ start}) \times \mathrm{backward}(1, c_k) \times P_{\mathrm{emiss}}(x_{1} | c_k).
 \end{eqnarray}

Using the backward trellis one can compute the likelihood simply as:

\begin{equation}
P(X=x) = \mathrm{backward}(0, \text{start}).
\end{equation}




In [71]:
example = train_sequences[1].x
example

['walk', 'walk', 'shop', 'clean']

In [72]:
log_backward, loglikelihood_b = hmm.log_backward_computations(example)
log_forward,  loglikelihood_f = hmm.log_forward_computations(example)

In [73]:
loglikelihood_b, loglikelihood_f

(-5.068232326005126, -5.068232326005127)

In [74]:
log_backward

array([[-4.41863845, -3.67819455, -2.65480569,        -inf],
       [-5.73879301, -3.88249502, -2.43166214, -0.98082925]])

In [75]:
state_pos = hmm.compute_state_posteriors(example)

In [76]:
state_pos.argmax(0)

array([0, 0, 1, 1])

## Use a HMM in the conll data

In [77]:
train_seq = corpus.read_sequence_list_conll(data_path + "/train-02-21.conll", 
                                            max_sent_len=100, max_nr_sent=5000)

In [78]:
ind_to_word  = {v: k for k, v in train_seq.x_dict.items()}
ind_to_state = {v: k for k, v in train_seq.y_dict.items()}
word_to_ind  = train_seq.x_dict
state_to_ind = train_seq.y_dict

In [79]:
X = []
Y = []
for i in range(len(train_seq)):
    xy = train_seq[i]
    X.append([ind_to_word[x_i] for x_i in xy.x])
    Y.append([ind_to_state[y_i] for y_i in xy.y])

In [80]:
X[1],Y[1]

(['Ms.', 'Haag', 'plays', 'Elianti', '.'],
 ['noun', 'noun', 'verb', 'noun', '.'])

In [81]:
hmm = HMM(word_to_ind, state_to_ind)
hmm.fit(X, Y)

/Users/davidbuchaca1/anaconda3/envs/py3_7/lib/python3.6/site-packages/ipykernel_launcher.py:85: RuntimeWarning: divide by zero encountered in log
/Users/davidbuchaca1/anaconda3/envs/py3_7/lib/python3.6/site-packages/ipykernel_launcher.py:86: RuntimeWarning: divide by zero encountered in log
/Users/davidbuchaca1/anaconda3/envs/py3_7/lib/python3.6/site-packages/ipykernel_launcher.py:87: RuntimeWarning: divide by zero encountered in log


In [82]:
y_hat = hmm.predict_labels(X[0])

### How to make a prediction

use `hmm.predict_labels ` to get the set of part of speech tags for a given sequence.

In [83]:
y_hat = hmm.predict_labels(X[0])

In [84]:
" ".join(X[0])

"In an Oct. 19 review of `` The Misanthrope '' at Chicago 's Goodman Theatre ( `` Revitalized Classics Take the Stage in Windy City , '' Leisure & Arts ) , the role of Celimene , played by Kim Cattrall , was mistakenly attributed to Christina Haag ."

In [85]:
" ".join(y_hat)

'adp det noun num noun adp . det noun . adp noun prt noun noun . . verb noun verb det noun adp noun noun . . noun conj noun . . det noun adp noun . verb adp noun noun . verb adv verb prt noun noun .'

In [86]:
result = skseq.sequences.sequence.Sequence(X[0],y_hat)

In [87]:
result

In/adp an/det Oct./noun 19/num review/noun of/adp ``/. The/det Misanthrope/noun ''/. at/adp Chicago/noun 's/prt Goodman/noun Theatre/noun (/. ``/. Revitalized/verb Classics/noun Take/verb the/det Stage/noun in/adp Windy/noun City/noun ,/. ''/. Leisure/noun &/conj Arts/noun )/. ,/. the/det role/noun of/adp Celimene/noun ,/. played/verb by/adp Kim/noun Cattrall/noun ,/. was/verb mistakenly/adv attributed/verb to/prt Christina/noun Haag/noun ./. 

### Measure accuracy in the train set

In [88]:
import tqdm

In [ ]:
Y_hat = []
for x in tqdm.tqdm(X):
    Y_hat.append(hmm.predict_labels(x))

 70%|███████   | 3517/5000 [00:37<00:13, 108.22it/s]

In [ ]:
correct = 0
total   = 0
for y,y_hat in zip(Y,Y_hat):
    for y_hat_k, y_k in zip(y,y_hat):
        total +=1
        if y_hat_k == y_k:
            correct +=1

In [ ]:
correct, total

In [ ]:
print("Accuracy posterior decode train data", correct/total)

### Measure accuracy in the test set

In [ ]:
len(test_seq)

In [ ]:
X_test = []
Y_test = []
for i in range(len(test_seq)):
    xy = train_seq[i]
    X_test.append([ind_to_word[x_i] for x_i in xy.x])
    Y_test.append([ind_to_state[y_i] for y_i in xy.y])

In [ ]:
Y_test_hat = []
for x in tqdm.tqdm(X_test):
    Y_test_hat.append(hmm.predict_labels(x))

In [ ]:
correct_test = 0
total_test   = 0
for y,y_hat in zip(Y_test,Y_test_hat):
    for y_hat_k, y_k in zip(y,y_hat):
        total_test +=1
        if y_hat_k == y_k:
            correct_test +=1

In [ ]:
print("Accuracy posterior decode test data", correct_test/total_test)